In [1]:
import os 
import warnings
import sys

import pandas as pd 
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import ElasticNet 

import mlflow 
import mlflow.sklearn

In [2]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2293,7.3,0.40,0.28,6.50,0.037,26.0,97.0,0.99148,3.16,0.58,12.6,7
2447,6.5,0.27,0.26,8.20,0.042,21.0,133.0,0.99612,3.43,0.64,9.8,6
1204,5.0,0.33,0.16,1.50,0.049,10.0,97.0,0.99170,3.48,0.44,10.7,6
4841,5.7,0.22,0.25,1.10,0.050,97.0,175.0,0.99099,3.44,0.62,11.1,6
279,7.0,0.30,0.49,4.70,0.036,17.0,105.0,0.99160,3.26,0.68,12.4,7
3660,5.8,0.32,0.28,4.30,0.032,46.0,115.0,0.98946,3.16,0.57,13.0,8
1132,6.8,0.39,0.31,14.35,0.043,28.0,162.0,0.99880,3.17,0.54,9.1,5
1863,6.6,0.28,0.30,7.80,0.049,57.0,202.0,0.99580,3.24,0.39,9.5,5
1167,7.4,0.23,0.43,1.40,0.044,22.0,113.0,0.99380,3.22,0.62,10.6,6
136,8.2,0.23,0.40,1.20,0.027,36.0,121.0,0.99200,3.12,0.38,10.7,6


In [10]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [3]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 127.0.0.1 --port 5000
remote_server_uri = "http://127.0.0.1:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env

In [4]:
mlflow.tracking.get_tracking_uri()


'http://127.0.0.1:5000'

In [5]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

INFO: 'ElasticNet_wine' does not exist. Creating a new experiment


In [6]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y

def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [7]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
Save to: mlruns/1/c1faf30f0f294e48a6724fbd3261b83d/artifacts


In [8]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.20224631822892092
Save to: mlruns/1/a141c2a344d94d6cae2bc7c9fcdf8771/artifacts


In [9]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlruns/1/6e99fd8d3b774ce2b69a1c42e20ab860/artifacts


In [11]:
from datetime import datetime
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiments = client.list_experiments() # returns a list of mlflow.entities.Experiment
print(experiments)

[<Experiment: artifact_location='mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>, <Experiment: artifact_location='mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>]


In [12]:
# get the run
_run = client.get_run(run_id="6e99fd8d3b774ce2b69a1c42e20ab860")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6112547988118587,
 'r2': 0.2157063843066196,
 'rmse': 0.7792546522251949}, params={'alpha': '0.1', 'l1_ratio': '0.1'}, tags={'mlflow.source.name': '/home/raunak314/miniconda3/envs/mle-jup/lib/python3.11/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'raunak314'}>, info=<RunInfo: artifact_uri='mlruns/1/6e99fd8d3b774ce2b69a1c42e20ab860/artifacts', end_time=1683829862042, experiment_id='1', lifecycle_stage='active', run_id='6e99fd8d3b774ce2b69a1c42e20ab860', run_uuid='6e99fd8d3b774ce2b69a1c42e20ab860', start_time=1683829861945, status='FINISHED', user_id='raunak314'>>


In [13]:
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)